### Imports

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

### Processing data

#### Assume our data is stored as a CSV and we have time stamps

#### For now I am generating fake data to test analysis on

In [9]:
# # Define features and data types
# features = {
#     "timestamp": pd.Timestamp,  # Qualitative (date and time)
#     # Gas sensors (V)
#     "GasSensor_0": np.float32,
#     "GasSensor_1": np.float32,
#     "GasSensor_2": np.float32,
#     "GasSensor_3": np.float32,
#     "GasSensor_4": np.float32,
#     "GasSensor_5": np.float32,
#     "GasSensor_6": np.float32,
#     "GasSensor_7": np.float32,
#     "GasSensor_8": np.float32,
#     "GasSensor_9": np.float32,
#     "GasSensor_10": np.float32,
#     "GasSensor_11": np.float32,
# 
#     "Temperature": np.float32,  # Quantitative (°C)
#     "Pressure": np.float32,    # Quantitative (Pa)
#     "Humidity": np.uint8,      # Quantitative (%RH)
#     "Battery_Charge": np.uint8,  # Quantitative (%)
#     "Net_Current": np.float32,  # Quantitative (A)
#     "Tension": np.float32,     # Quantitative (V)
#     "Battery_Temp": np.float32, # Quantitative (°C)
# 
#     # Particulate matter (µg/m³)
#     "PM1.0": np.float32,
#     "PM2.5": np.float32,
#     "PM4.0": np.float32,
#     "PM10.0": np.float32,
# 
#     # Particulate matter (1/cm³)
#     "PM1.0_cm3": np.float32,
#     "PM2.5_cm3": np.float32,
#     "PM4.0_cm3": np.float32,
#     "PM10.0_cm3": np.float32,
# 
#     "PM_Typ_Size": np.float32,  # Quantitative (µm)
#     "CO2": np.float32,           # Quantitative (ppm)
# }

# Generate random data
num_datapoints = 100 

# temp = pd.Timestamp(year=2017, month=1, day=1, hour=12, second=0)

# add more columns later
fake_df = pd.DataFrame(columns=['Temperature', 'Pressure', 'Humidity', 'Battery_Charge', 'Net_Current', 'Tension'])

start_time = datetime(2024, 2, 20, 10, 0, 0)  # Adjust start date and time

# the idea is to create random data right now. Tighten up the range so it's only dealing with a particular kind of 
# weather for now. Some combinations are thermodynamically impossible if you arbitrarily choose property values
for i in range(num_datapoints):
    print('test')
    # fake_df['Temperature'] = random.uniform(0, 100) # degrees celsius
    # fake_df['Pressure'] = random.uniform(99284.51, 103421) # potential pressure values in earth
    # fake_df['Humidity'] = 
    # fake_df['Battery_Charge'] =
    # fake_df['Net_Current'] =
    # fake_df['Tension'] = 

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


#### Visualizing data

### Visualizing data